In [1]:
import torch, json, pathlib
from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

batch_size = 1
guidance_scale = 8
prompt = "the simplest rectangle"

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=16,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

render_mode = 'stf' # you can change this to 'stf'
size = 32

  0%|          | 0/16 [00:00<?, ?it/s]

In [4]:
cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

c:\Users\gdste\OneDrive - University of Patras\Uni\Diploma Thesis\sterg_3d_twin\mcp_llm\mcp-server-demo\shap-e-local\shap_e\models\stf\renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
c:\Users\gdste\OneDrive - University of Patras\Uni\Diploma Thesis\sterg_3d_twin\mcp_llm\mcp-server-demo\shap-e-local\shap_e\models\stf\renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(


HTML(value='<img src="data:image/gif;base64,R0lGODlhIAAgAIUAAHGnwm6guWyet22ds2qdtWibtGmZsWmYsGaWrGaUqWOSp2OQqG…

In [9]:
# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    t = decode_latent_mesh(xm, latent).tri_mesh()
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        t.write_ply(f)
    with open(f'example_mesh_{i}.obj', 'w') as f:
        t.write_obj(f)


In [7]:
from PIL import Image

# Save each rendered image as a PNG file
for idx, img in enumerate(images):
    img.save(f"texture_{idx}.png")

In [6]:
import os
print(os.getcwd())

c:\Users\gdste\OneDrive - University of Patras\Uni\Diploma Thesis\sterg_3d_twin\mcp_llm\mcp-server-demo\shap-e-local
